<a href="https://colab.research.google.com/github/Angelina151612/Sentiment-analysis-with-tweets/blob/br1/Source/clean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re

import nltk.corpus
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import pandas as pd

nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
def open_file(path):
      with open(path, encoding="ISO-8859-1") as readfile:
          return pd.read_csv(
              readfile,
              engine="python",
              error_bad_lines=False,
              header=None,
          )

In [3]:
def get_data():
    path = "/content/drive/MyDrive/Colab Notebooks/Sentiment analysis with tweets/Data/data.csv"
    df = open_file(path)
    return df[5], df[0]

In [4]:
def normalization(data):
    data = data.str.lower()
    data = data.apply(lambda tweet: re.sub(r"@[A-Za-z0-9]+", "", tweet))  # delete names
    data = data.apply(lambda tweet: re.sub(r"https?://[^\s>]+", "", tweet))  # delete url
    data = data.apply(lambda tweet: re.sub(r"\d+", "", tweet))  # delete numbers
    data = data.apply(lambda tweet: re.sub(r"[^0-9A-Za-z \t]", "", tweet))  # delete punctuation
    return data

***Stop words*** are those words in natural language that have a very little meaning, such as "is", "an", "the", etc. 

In [5]:
def delete_stop_words(data):
    stop_words = stopwords.words("english")
    data = data.apply(lambda tweet: [word for word in tweet if word not in (stop_words)])
    return data

In [6]:
def tokenization(data):
    return data.apply(lambda tweet: tweet.split())

We must leave at least two repeated letters because there are words with double letters. The main idea was to make spelling correction process faster. It didn't help, but some words with same mistakes became one in our dictionary.

-> noooo, nooooooo: noo

In [7]:
def remove_repeated_letters(data):
    pattern = re.compile(r"(.)\1{2,}")
    return data.apply(lambda tweet: [pattern.sub(r"\1\1", word) for word in tweet])

***Lemmatization*** is the process of grouping together the different inflected forms of a word so they can be analysed as a single item.

-> help, helping, helped : help

In [8]:
def lemmatizaton(data):
    lemmatizer = WordNetLemmatizer()
    return data.apply(lambda tweet: [lemmatizer.lemmatize(word) for word in tweet])

In [9]:
def save_prepared_data(data, marks):
    data_to_save = data.apply(lambda tweet: " ".join(tweet))
    df = pd.DataFrame({"tweet": data_to_save, "mark": marks})
    df = df[df["tweet"].notna()]
    df.to_csv("/content/drive/MyDrive/Colab Notebooks/Sentiment analysis with tweets/Data/prepared_data.csv", index=False)

In [17]:
x, y = get_data()
before = x[9146:9149]
pd.set_option("display.max_colwidth", None)

Skipping line 912304: unexpected end of data


In [18]:
x = normalization(x)
x = tokenization(x)
x = delete_stop_words(x)
x = remove_repeated_letters(x)
x = lemmatizaton(x)

In [36]:
for i, j, k in zip(before.tolist(),x[9146:9149].tolist(), range(len(before))):
  print('\033[1m\nExample', k+1, '\033[0m')
  print('Before:\n', i, '\nAfter:\n', j)


Example 1 
Before:
 @dante321 http://twitpic.com/3hqos - I have to agree with supermom...YOU POOR THING!! What we do for love  
After:
 ['agree', 'supermomyou', 'poor', 'thing', 'love']

Example 2 
Before:
 Will tweet more diligently after exams. Back to studying  
After:
 ['tweet', 'diligently', 'exam', 'back', 'studying']

Example 3 
Before:
 reeeeeeeeeealy wants some Mickey D's dollar menu cheeseburger. I'm huuuuuuuuunnnnnngggggrrrrryyyyy  
After:
 ['reealy', 'want', 'mickey', 'd', 'dollar', 'menu', 'cheeseburger', 'im', 'huunnggrryy']


In [ ]:
save_prepared_data(x, y)